### Packages

In [ ]:
## packages
import math
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split 
from scipy.stats import linregress
from joblib import Parallel, delayed
import psutil
import re
import torch
import botorch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from botorch.models.model import Model
from botorch.posteriors.posterior import Posterior
from botorch.acquisition import LogExpectedImprovement, PosteriorMean
from botorch.acquisition.analytic import LogProbabilityOfImprovement
from botorch.generation.gen import gen_candidates_torch
from botorch.optim.optimize import optimize_acqf

### Real-Time Data Processing

In [ ]:
# Natural sorting 
def natural_sort_key(s):
    _nsre = re.compile('([0-9]+)')
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]  
## set of function to read and process the raw spectra
def read_files(loc):
    files_all = sorted(os.listdir(loc), key = natural_sort_key)
    ABS_names = []
    PL_names = []
    ABS_all = []
    PL_all = []
    for file in files_all:
        if (file.startswith("Abs")):
            ABS = pd.read_csv(loc + "//" + file , header = None).to_numpy()
            ABS_all.append(ABS)
            ABS_names.append(file.split(".")[0])
        elif (file.startswith("PL")):
            PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
            PL_all.append(PL)
            PL_names.append(file.split(".")[0])
        elif(file.startswith("Wavelength_Abs")):
            WL_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("Wavelength_PL")):
            WL_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_Abs")):
            DR_ABS = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("DR_PL")):
            DR_PL = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("LR")):
            LR = pd.read_csv(loc + "//" + file, header = None).to_numpy()
        elif(file.startswith("FR")):
            FR = pd.read_csv(loc + "//" + file, header = None).to_numpy()    
    return WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR
def idx_min(y, x):
    diff = np.abs(y[:, 0] - x)
    idx = diff.argmin()
    return idx
# extract reactive phase
def extract(x, idx1, idx2, numFirstElements):
    x_mean = x[idx1:idx2 + 1, :].mean(axis = 0)
    x_sort = np.sort(x_mean)
    idx_phase = np.nonzero(np.in1d(x_mean, x_sort[:numFirstElements]))[0]
    x_phase = x[:, idx_phase]
    return x_phase
def spectra_avg(x):
    x_avg = x.mean(axis = 1) # avg over extracted reactive phase spectra for each WL; will result in a row matrix
    x_avg = x_avg[:, np.newaxis]
    return x_avg
def baseline_zero(x, idx_low, idx_high):
    x_baseline_zero = x - x[idx_low:idx_high + 1, :].mean() # make baseline zero; subtracting mean of PL at LL - HL nm
    return x_baseline_zero
def linear_int_x(y, x, i, y_btw):
    x_btw = x[i, 0] - ((x[i + 1, 0] - x[i, 0]) / (y[i + 1, 0] - y[i, 0])) * (y[i, 0] - y_btw)
    return x_btw
def linear_int_y(y, x, i, x_btw):
    y_btw = y[i, 0] - ((y[i + 1, 0] - y[i, 0]) / (x[i + 1, 0] - x[i, 0])) * (x[i, 0] - x_btw)
    return y_btw
def peak_info(y, x, emission_intensity):
    # correction for emission peak intensity in case min is not exactly zero
    # min_intensity = y.min()
    min_intensity = 0 # without correction
    intensity_peak = emission_intensity - min_intensity
    # emission peak area
    area_peak = np.trapz(y.T, x = x.T)[0]
    # emission peak intensity and area
    output = [intensity_peak, area_peak]
    return np.array(output)[:, np.newaxis]
# process ABS
def spectra_extract_ABS(WL, DR, LR, ABS_all, ABS_names):
    # baseline WL limits
    baseline_LL = 700
    baseline_HL = 800
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    lastPeak_LL = 250
    lastPeak_HL = 350 
    idx_lastPeak_LL = idx_min(WL, lastPeak_LL)
    idx_lastPeak_HL = idx_min(WL, lastPeak_HL)
    # excitation WL info
    excitation_WL = 300
    idx_excitationWL = idx_min(WL, excitation_WL)
    # initiate files to be exported
    WLABS_processed = WL
    ABS_excitationWL_list = []
    # ABS_excitationWL_list = np.array([["Absorbance at excitation WL"]])
    for (item, name) in zip(ABS_all, ABS_names):
        ABS_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 50
        ABS_phase = extract(ABS_only, idx_lastPeak_LL, idx_lastPeak_HL, param)
        ABS_phase_avg = spectra_avg(ABS_phase) 
        arg_log = (ABS_phase_avg - DR) / (LR - DR) # Beer-Lambert law
        ABS_processed = - np.log10(arg_log) # Beer-Lambert law
        ABS_processed_baseline = baseline_zero(ABS_processed, idx_WL_LL, idx_WL_HL) 
        WLABS_processed = np.concatenate( [WLABS_processed, ABS_processed_baseline] , axis = 1) # attach WL and processed ABS
        # WLABS_processed_filtered = WLABS_processed[np.isnan(WLABS_processed[:, 1]) == False] # remove NaN ABS
        ABS_excitationWL = linear_int_y(ABS_processed_baseline, WL, idx_excitationWL, excitation_WL)
        ABS_excitationWL_list.append(ABS_excitationWL)
        # ABS_excitationWL_list = np.concatenate( [ABS_excitationWL_list, np.array([ABS_excitationWL])[:, np.newaxis]] , axis = 1)
    ABS_excitationWL_list = np.array(ABS_excitationWL_list)[:, np.newaxis]
    # return WLABS_processed, ABS_excitationWL_list # processed ABS spectra as well as the Abs300nm as one one of the optical features of interest
    return ABS_excitationWL_list
# process PL
def spectra_extract_PL(WL, DR, PL_all, PL_names):
    ## baseline WL limits
    baseline_LL = 800
    baseline_HL = 1000
    idx_WL_LL = idx_min(WL, baseline_LL)
    idx_WL_HL = idx_min(WL, baseline_HL)
    # to extract reactive phase
    excWL_LL = 250
    excWL_HL = 350
    idx_excWL_LL = idx_min(WL, excWL_LL)
    idx_excWL_HL = idx_min(WL, excWL_HL)
    ## emission peak WL and the WL range for the right side of emission peak
    emPeak_WL = 446
    emPeak_LL = 345
    emPeak_HL = 700
    idx_emPeak_LL = idx_min(WL, emPeak_LL)
    idx_emPeak_HL = idx_min(WL, emPeak_HL)
    WL_aroundPeak = WL[idx_emPeak_LL: idx_emPeak_HL + 1, :]
    idx_intensityWL = idx_min(WL, emPeak_WL) # index for emission peak WL
    ## initiate files to be exported
    WLPL_processed = WL
    emission_details_list = []
    # emission_details = np.array([["Emission Peak WL"], ["Emission Peak Intensity"], ["FWHM"], ["Emission Peak Area"]])
    for (item, name) in zip(PL_all, PL_names):
        PL_only = item[1:, :]
        ## extract reactive phase and remove carrier phase (PFO)
        param = 50
        PL_phase = extract(PL_only, idx_excWL_LL, idx_excWL_HL, param) 
        PL_phase_avg = spectra_avg(PL_phase)
        PL_processed = PL_phase_avg - DR
        PL_processed_baseline = baseline_zero(PL_processed, idx_WL_LL, idx_WL_HL) 
        WLPL_processed = np.concatenate( [WLPL_processed, PL_processed_baseline] , axis = 1) # attach WL and processed PL
        ## emission peak info
        PL_emissionWL = linear_int_y(PL_processed_baseline, WL, idx_intensityWL, emPeak_WL)
        PL_aroundPeak = PL_processed_baseline[idx_emPeak_LL: idx_emPeak_HL + 1, :]
        info_peak = peak_info(PL_aroundPeak, WL_aroundPeak, PL_emissionWL)
        emission_details_list.append(info_peak)
        # emission_details = np.concatenate( [emission_details, info_peak_method] , axis = 1)
    emission_details_list = np.concatenate(emission_details_list, axis = 1).T
    # return WLPL_processed, emission_details_list # processed PL spectra as well as PLI and PLA as two of the optical features of interest
    # return emission_details_list # emission peak intensity and area as optical features of interest
    return emission_details_list[:, [1]] # emission peak area as one of the output parameters of interest to calculate relative PLQY
## calculate relative PLQY
def rel_PLQY(y_abs, y_area):
    # Convert y_abs and y_area to pytorch tensors if they are numpy arrays
    if not torch.is_tensor(y_abs):
        y_abs = torch.from_numpy(y_abs.astype(np.float32))
    if not torch.is_tensor(y_area):
        y_area = torch.from_numpy(y_area.astype(np.float32))
    # data related to the dye and solvent
    PLQY_ref = 0.1336
    ABS_ref = 0.9115
    area_ref = 198493.1521
    # calculate relative_PLQY
    PLQY_sample = PLQY_ref * (y_area / area_ref) * ((1 - torch.pow(torch.tensor(10), -ABS_ref)) / (1 - torch.pow(torch.tensor(10), -y_abs)))
    return PLQY_sample

### ML Modeling

In [ ]:
## scale output
def non_dim_y(y_initial):
    # Convert y_initial to pytorch tensor if it isn't
    if not torch.is_tensor(y_initial):
        y_initial = torch.from_numpy(y_initial.astype(np.float32))
    # Get min and max info
    min_info, min_idx = torch.min(y_initial, axis=0, keepdim=True)
    max_info, max_idx = torch.max(y_initial, axis=0, keepdim=True)
    statNorm_info = torch.concatenate((min_info, max_info), axis=0)
    # Nondimensionalize the y's
    y_scaled = (y_initial - min_info) / (max_info - min_info)
    return y_scaled, statNorm_info
def dim_y(y_transform, statNorm_info):
    # Get min and max info
    min_info = torch.index_select(statNorm_info, dim=0, index=torch.tensor([0]))
    max_info = torch.index_select(statNorm_info, dim=0, index=torch.tensor([1]))
    # Give y's dimension
    y_scaled = min_info + y_transform * (max_info - min_info)
    return y_scaled
## nondimensionalize inputs
def non_dim_x(x_initial):
    # For doping project
    x_normalized = np.zeros((x_initial.shape[0], x_initial.shape[1] - 3))
    x_normalized[:, 0] = (x_initial[:, 0] - 120) / (180 - 120) # x1 = temperature
    x_normalized[:, 1] = (x_initial[:, 1] - 80) / (120 - 80) # x2 = CuI
    x_normalized[:, 2] = (x_initial[:, 2] - 80) / (120 - 80) # x3 = ZnI2
    x_normalized[:, 3] = (x_initial[:, 3] - 20) / (80 - 20) # x4 =  Cs-Oleate
    x_normalized[:, 4] = (x_initial[:, 4] - 220) / (280 - 220) # x5 = ODE
    return x_normalized
## split dataset to training and validation
def split_set(x, y, train_ratio = 0.8, seed_num = 4862): # important hyper-parameter
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = train_ratio, random_state = seed_num)
    return x_train, x_test, y_train, y_test
## ENN structure
def structure_info(num_models, layer_min = 8, layer_max = 10, node_min = 20, node_max = 40, seed_num = 100): # should tune these hyperparameters
    random.seed(seed_num)
    detail_all = []
    for i in range (num_models):
        num_layers = random.randint(layer_min, layer_max)
        nodes_list = []
        for j in range(num_layers):
            num_nodes = random.randint(node_min, node_max)
            nodes_list.append(num_nodes)
        detail = [num_layers, nodes_list]
        detail_all.append(detail)
    return detail_all
# Create Neural Network model
class NeuralNetwork(nn.Module):
    def __init__(self, structure, network_type, input_shape, output_shape):
        super(NeuralNetwork, self).__init__() # Initialize the network object using the superclass constructor
        self.layers = nn.ModuleList() #List of layers
        self.network_type = network_type
        input_size = input_shape[-1] # Last dimension stores num of input features
        # If network is a feed-forward neural network
        if self.network_type == "ff":
            for num_layers, nodes_list in structure:
                for num_nodes in nodes_list:
                    self.layers.append(nn.Linear(input_size, num_nodes)) # Apply the linear layer
                    self.layers.append(nn.ReLU()) # Apply the ReLU activation
                    input_size = num_nodes #reset input size to be the number of nodes determined by structure_info function
            self.layers.append(nn.Linear(input_size, output_shape[-1]))  # final layer outputs predicted y's
        # If network is a cascade neural network
        elif self.network_type == "cascade":
            for num_layers, nodes_list in structure:
                for num_nodes in nodes_list:
                    self.layers.append(nn.Linear(input_size, num_nodes)) # Apply the linear layer
                    self.layers.append(nn.ReLU()) # Apply the ReLU activation
                    input_size += num_nodes #reset input size to be the number of nodes determined by structure_info function
            self.output_layer = nn.Linear(input_size, output_shape[-1]) # number of output parameters
    def forward(self, x):
        # If network is a feed-forward neural network
        if self.network_type == "ff": 
            for layer in self.layers:
                x = layer(x)          
            return x
        # If network is a cascade neural network
        elif self.network_type == "cascade":
            outputs = x
            for layer in self.layers:
                if isinstance(layer, nn.Linear):
                    x = layer(outputs)  # Apply the linear layer
                elif isinstance(layer, nn.ReLU):
                    x = layer(x)  # Apply the ReLU activation
                    outputs = torch.cat((outputs, x), dim = outputs.ndim-1) # Concatenate the current output to all previous ReLU outputs, along the last dimension
            x = self.output_layer(outputs)     
            return x
# Early Stopping
class EarlyStopping:
    def __init__(self, patience, min_delta):
        self.patience = patience
        self.min_delta = min_delta
        self.patience_counter = 0
        self.best_model_state = None
        self.min_validation_loss = float('inf')
    def should_stop_early(self, model, current_validation_loss):
        # Reset counter if new minimum validation loss found
        if current_validation_loss < self.min_validation_loss:
            self.min_validation_loss = current_validation_loss
            self.best_model_state = model.state_dict()
            self.patience_counter = 0
        # If validation loss is greater than minimum, increase the patience counter
        elif current_validation_loss > (self.min_validation_loss + self.min_delta):
            self.patience_counter += 1
            if self.patience_counter >= self.patience:
                return True
        return False
    def get_best_model_state(self):
        return self.best_model_state
## Training the neural network using PyTorch
def parallel_train_ML_pytorch(train_x, train_y, val_x, val_y, structure, network_type, batch_size=8, num_epochs=3000, learning_rate=1e-4, patience=500, min_delta=1e-4):
    # Convert dataset to tensors if they are numpy arrays
    if not torch.is_tensor(train_x):
        train_x = torch.from_numpy(train_x.astype(np.float32))
    if not torch.is_tensor(train_y):
        train_y = torch.from_numpy(train_y.astype(np.float32))
    if not torch.is_tensor(val_x):
        val_x = torch.from_numpy(val_x.astype(np.float32))
    if not torch.is_tensor(val_y):
        val_y = torch.from_numpy(val_y.astype(np.float32))
    # Instantiate training and test data
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(val_x, val_y)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=len(val_y), shuffle=False)
    # Create model, optimizer, early stopping class, and loss function 
    model = NeuralNetwork(structure=structure, network_type=network_type, input_shape=train_x.shape, output_shape=train_y.shape)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #lr - learning step
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    criterion = nn.MSELoss()
    # Data to store during training
    history = {'epoch': [], 'train_loss': [], 'val_loss': []}
    epochs = []
    train_loss_list = []
    val_loss_list = []
    # Number of outputs to train
    num_outputs = train_y.shape[-1]
    # Train the NN
    for epoch in range(num_epochs):
        ## Model Training
        model.train()
        running_loss = 0.0
        epochs.append(epoch)
        for inputs, labels in train_loader:
            # Reset optimizer
            optimizer.zero_grad()
            # Feed inputs
            outputs = model(inputs)
            # Loss function is the sum of output losses
            loss = None
            for train_output_idx in range(num_outputs):
                train_output = torch.index_select(outputs, dim=outputs.ndim-1, index=torch.tensor(train_output_idx))
                train_label = torch.index_select(labels, dim=labels.ndim-1, index=torch.tensor(train_output_idx))
                if loss is None:
                    loss = criterion(train_output, train_label)
                else:
                    loss += criterion(train_output, train_label)
            # Back propagation
            loss.backward()
            # Update optimizer
            optimizer.step()
            # Add loss to accumulated loss 
            running_loss += loss.item()
        # Compute average loss per batch
        train_loss = running_loss / len(train_loader)
        train_loss_list.append(train_loss)
        ## Model Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            # Validate the NN
            for val_inputs, val_labels in test_loader:
                val_outputs = model(val_inputs)
                val_loss = None
                # Validation loss is the sum of output losses
                for val_output_idx in range(num_outputs):
                    val_output = torch.index_select(val_outputs, dim=val_outputs.ndim-1, index=torch.tensor(val_output_idx))
                    val_label = torch.index_select(val_labels, dim=val_labels.ndim-1, index=torch.tensor(val_output_idx))
                    if val_loss is None:
                        val_loss = criterion(val_output, val_label).item()
                    else:
                        val_loss += criterion(val_output, val_label).item()
            # Compute average loss per batch
            val_loss /= len(test_loader)
            val_loss_list.append(val_loss)
        #print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}')
        if early_stopping.should_stop_early(model, val_loss):
            #print(f'Early stopping triggered at epoch {epoch + 1}')
            break
    # Load the best model
    if early_stopping.get_best_model_state() is not None:
        model.load_state_dict(early_stopping.get_best_model_state())
    # Gather history
    history['epoch'] = epochs
    history['train_loss'] = train_loss_list
    history['val_loss'] = val_loss_list
    result = [model, history]
    return result
## deconvolute the model and training history
def deconvolute_ML(ensemble_result):
    ensemble_model = []
    ensemble_history = []
    for element in ensemble_result:
        ensemble_model.append(element[0])
        ensemble_history.append(element[1])
    return ensemble_model, ensemble_history
## plots related to the training process
def learn_plot(history):
    fig = plt.figure()
    # loss values - MSE
    plt.plot(history['train_loss'], color = 'blue')
    plt.plot(history['val_loss'], color = 'red')
    plt.title('Loss (MSE) - Epoch')
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch Number')
    plt.legend(['Training', 'Validation'])
    fig.tight_layout()
    plt.show()
## Plot for predicted vs. true outputs - for individual models
def predict_plot_separate(model, x, y_true):
    if not torch.is_tensor(x):
        x = torch.from_numpy(x.astype(np.float32))
    dim = y_true.shape[-1]
    plt.figure(figsize = (3.5 * dim, 3.5), dpi = 200)
    # Plot each output separately
    for i in range(dim):
        y_true_i = y_true[:,i]
        model_prediction = model(x).detach().numpy()
        y_predicted_i = model_prediction[:, i]
        detail = linregress(y_true_i, y_predicted_i)
        plt.subplot(1, dim, i + 1)
        plt.scatter(y_true_i, y_predicted_i, color = 'blue', alpha = 0.8, label = 'y' + str(i) + ': $R^2 = $' + str(round((detail.rvalue ** 2), 4)))
        plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
        # plt.title('y' + str(i + 1))
        plt.ylabel('Predicted')
        plt.xlabel('Actual')
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.legend() 
    plt.tight_layout()
    plt.show()
## plot related to the predicted vs. true outputs - for the entire ENN 
def predict_plot_mean(ensemble_NN, x, y_true):
    if not torch.is_tensor(x):
        x = torch.from_numpy(x.astype(np.float32))
    y_pred = []
    for model in ensemble_NN:
        y_pred.append(model(x).detach().numpy())
    y_pred_arr = np.concatenate(y_pred, axis = 1)
    dim = y_true.shape[1]
    y_pred_mean = []
    y_pred_std = []
    plt.figure(figsize = (3.5 * dim, 3.5), dpi = 200)
    # Plot each output separately
    for i in range(dim):
        element = y_pred_arr[:, i::dim]
        element_mean = np.mean(element, axis = 1)
        element_std = np.std(element, axis = 1)
        element_detail = linregress(y_true[:, i], element_mean)
        y_pred_mean.append(element_mean[:, np.newaxis])
        y_pred_std.append(element_std[:, np.newaxis])
        plt.subplot(1, dim, i + 1)
        plt.errorbar(y_true[:, i], element_mean, yerr = element_std, fmt = 'o', color = 'blue', alpha = 0.8, markersize = 5, label = 'y' + str(i) + ': $R^2 = $' + str(round((element_detail.rvalue ** 2), 4)))
        plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
        # plt.title('y' + str(i + 1))
        plt.ylabel('Predicted')
        plt.xlabel('Actual')
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.legend()
    plt.tight_layout()
    pred_mean = np.concatenate(y_pred_mean, axis = 1)
    pred_std = np.concatenate(y_pred_std, axis = 1)
    return pred_mean, pred_std
# Predict relative PLQY and compare it to true relative PLQY
def predict_plot_mean_PLQY(ensemble_NN, x, y_true, statNorm_info):
    if not torch.is_tensor(x):
        x = torch.from_numpy(x.astype(np.float32))
    rel_PLQY_pred = []
    for model in ensemble_NN:
        predictions = model(x).detach()
        y_pred_dim = dim_y(predictions, statNorm_info)
        rel_PLQY_pred.append(rel_PLQY(y_pred_dim[:,[0]], y_pred_dim[:,[1]]).numpy())
    rel_PLQY_pred_arr = np.concatenate(rel_PLQY_pred, axis = 1)
    plt.figure(figsize = (3.5, 3.5), dpi = 200)
    rel_PLQY_pred_mean = np.mean(rel_PLQY_pred_arr, axis = 1)
    rel_PLQY_pred_std = np.std(rel_PLQY_pred_arr, axis = 1)
    y_true_dim = dim_y(y_true, statNorm_info)
    rel_PLQY_true = rel_PLQY(y_true_dim[:,0], y_true_dim[:,1]).numpy()
    element_detail = linregress(rel_PLQY_true, rel_PLQY_pred_mean)
    plt.errorbar(rel_PLQY_true, rel_PLQY_pred_mean, yerr = rel_PLQY_pred_std, fmt = 'o', color = 'blue', alpha = 0.8, markersize = 5, label = '$R^2 = $' + str(round((element_detail.rvalue ** 2), 4)))
    plt.plot([0, 1],[0, 1], color = 'red', label = 'Parity')
    plt.title('Relative PLQY')
    plt.ylabel('Predicted')
    plt.xlabel('True')
    plt.xlim([0.0, 0.25])
    plt.ylim([0.0, 0.25])
    plt.legend()
    plt.tight_layout()
    return rel_PLQY_pred_mean, rel_PLQY_pred_std
## save the ENN that's been trained
def save_ENN(ENN, path):
    for n, model in enumerate(ENN):
        model_path = os.path.join(path, str(n))
        os.mkdir(model_path)
        torch.save(model, os.path.join(model_path , "model" + str(n)))
## load the ENN that's been saved
def load_ENN(path):
    model_idx = sorted(os.listdir(path), key = len)
    ensemble = []
    for idx in model_idx:
        model_path = os.path.join(path, idx)
        ensemble.append(torch.load(os.path.join(model_path, "model" + idx)))
    return ensemble

### Optimization

In [ ]:
# During optimization, this function is necessary to handle the case where the model output tensor has three dimensions, i.e. (len(X), 1, 2)
def get_relative_PLQY(y):
    # calculate relative_PLQY    
    y_abs = torch.index_select(y, dim=y.ndim-1, index=torch.tensor([0]))
    y_area = torch.index_select(y, dim=y.ndim-1, index=torch.tensor([1]))
    PLQY_sample = rel_PLQY(y_abs, y_area)
    return PLQY_sample
class ENN_Posterior(Posterior):
    # Abstract property device
    device = torch.device('cpu')
    # Abstract property dtype
    dtype = torch.float32
    # Abstract property event_shape (the shape of a single sample)
    event_shape = torch.Size()
    def __init__(self, X, ensemble_py, statNorm_info, optimize_options):
        self.X = X
        self.ensemble_py = ensemble_py
        # Get the predictions for each model in the ensemble
        outputs = torch.tensor([])
        for model in ensemble_py:
            y_nondim = model(X)
            y_dim = dim_y(y_nondim, statNorm_info) # Dimensionalize the outputs
            y_pred = get_relative_PLQY(y_dim) # Calculate relative PLQY
            if (optimize_options[0] == "minimize"):
                y_target = optimize_options[1]
                obj_fun = torch.abs(y_target - y_pred) 
            else:
                obj_fun = y_pred
            # Concatenate the objective function to the outputs tensor
            outputs = torch.cat((outputs, obj_fun), dim=1)
        # Calculate and store the mean of ensemble predictions
        torch_mean = torch.mean(outputs, dim=1)
        self.mean = torch_mean
        # Calculate and store the variance of ensemble predictions
        torch_var = torch.var(outputs, dim=1)
        self.variance = torch_var
    def rsample(self, sample_shape=None, base_samples=None):
        # Construct a normal distribution and sample from it
        mu = self.mean
        sigma = self.variance
        dist = torch.normal(mu, sigma) 
        return dist
class ENN_Model(Model):
    num_outputs=None #need to have num_outputs defined or else error is thrown
    def __init__(self, ensemble_py, statNorm_info, optimize_options, num_objectives, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.ensemble_py = ensemble_py
        self.statNorm_info = statNorm_info
        self.optimize_options = optimize_options
        self.num_outputs = num_objectives
    # Computes the posterior over the model output at the provided points.
    def posterior(self, X, output_indices=None, observation_noise=False, posterior_transform=None, **kwargs):
        return ENN_Posterior(X, self.ensemble_py, self.statNorm_info, self.optimize_options)  

In [ ]:
# used of expected improvement (EI)
def best_data(y_abs, y_area, optimize_options):
    obj_fun_data = rel_PLQY(y_abs, y_area)
    if optimize_options[0] == "maximize":
        obj_fun_best = obj_fun_data.max()
    elif optimize_options[0] == "minimize":
        y_target = optimize_options[1]
        obj_fun_best = torch.abs(y_target - obj_fun_data).min()
    return obj_fun_best
## aquisition functions
def botorch_optimize(ensemble, optimize_options, policy, obj_fun_best, input_dim, statNorm_info, num_objectives=1, num_restarts=100, num_samples=500000):
    trained_model = ENN_Model(ensemble, statNorm_info, optimize_options, num_objectives)
    maximize = (True if optimize_options[0] == "maximize" else False)
    if policy == "EI":
        acqf = LogExpectedImprovement(trained_model, best_f=obj_fun_best, maximize=maximize)
    elif policy == "PI":
        acqf = LogProbabilityOfImprovement(trained_model, best_f=obj_fun_best, maximize=maximize)
    elif policy == "EPLT":
        acqf = PosteriorMean(trained_model, maximize=maximize)
    else:
        print('The policy type is invalid!')
        return None, None
    bounds = torch.tensor([[0. for i in range(input_dim)], [1. for i in range(input_dim)]])
    x_best, best_acq_value = optimize_acqf(acqf, bounds, q=1, num_restarts=num_restarts, raw_samples=num_samples, gen_candidates=gen_candidates_torch, return_best_only=True)
    y_best = trained_model.posterior(x_best).mean # The ensemble mean is the prediction
    x_best = x_best.detach().numpy() # Remove the gradient info and convert to numpy
    y_best = y_best.detach().numpy() # Remove the gradient info and convert to numpy
    return x_best, y_best
## optimization
def exp_sel_botorch(y, FR, optimize_options, policy, network_type="cascade", batch_size = 8, num_epochs=3000, learning_rate=1e-4, patience=500, min_delta=1e-4, num_models=20, num_objectives=1, num_restarts=100, num_samples=500000):
    ## dataset and preprocessing needed to be able to train the model
    y_norm, statNorm_info = non_dim_y(y)
    # Get best objective function seen for optimization
    obj_fun_best = best_data(y[:, [0]], y[:,[1]], optimize_options)
    x_norm = non_dim_x(FR)
    x_train, x_valid, y_train, y_valid = split_set(x_norm, y_norm)
    ## train the ENN
    start_ML = time()
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    architecture = structure_info(num_models)
    result_aggregate_ML = Parallel(n_jobs = -2)(delayed(parallel_train_ML_pytorch)(x_train, y_train, x_valid, y_valid, [architecture[i]], network_type=network_type, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate, patience=patience, min_delta=min_delta) for i in range(num_models))
    ensemble, history = deconvolute_ML(result_aggregate_ML)
    end_ML = time()
    print("------------------------------------------------------------------------------------------------------------")
    print(f"The execution time to train the ensemble NN is {str(end_ML - start_ML)} seconds.")
    print("------------------------------------------------------------------------------------------------------------")
    ## optimization and achive the best input variables needed for the future experiments
    start_opt = time()
    input_dim = x_norm.shape[-1]
#     p = psutil.Process()
#     p.nice(psutil.REALTIME_PRIORITY_CLASS)
    x_best, y_best = botorch_optimize(ensemble, optimize_options, policy, obj_fun_best, input_dim, statNorm_info, num_objectives=num_objectives, num_restarts=num_restarts, num_samples=num_samples)    
    end_opt = time()
    print("------------------------------------------------------------------------------------------------------------")
    print(f"The execution time for optimization is {str(end_opt - start_opt)} seconds.")
    print("------------------------------------------------------------------------------------------------------------")
    return x_best, y_best, ensemble, history, x_train, y_train, x_valid, y_valid, architecture, obj_fun_best, statNorm_info

### Master Code

In [ ]:
def count_files(path):
    files_all = os.listdir(path)
    count_ABS = 0
    count_PL = 0
    for file in files_all:
        if (file.startswith("Abs")):
            count_ABS = count_ABS + 1
        elif (file.startswith("PL")):
            count_PL = count_PL + 1
    return count_ABS, count_PL
def x_dim(x_norm, FR, path):
    ## for doping project; should be updated later
    x_final = np.zeros((x_norm.shape[0], x_norm.shape[1] + 3))
    x_final[:, 0] = (x_norm[:, 0] * (180 - 120)) + 120
    x_final[:, 1] = (x_norm[:, 1] * (120 - 80)) + 80
    x_final[:, 2] = (x_norm[:, 2] * (120 - 80)) + 80
    x_final[:, 3] = (x_norm[:, 3] * (80 - 20)) + 20
    x_final[:, 4] = (x_norm[:, 4] * (280 - 220)) + 220
    # last three columns
    x_final[:, -3] = 0.5 * np.sum(x_final[:, 1:5], axis = 1) # x6 = PFO
    x_final[:, -2] = 2 # x7 = 2 always; wait for 2 residence times then record the spectra
    x_final[:, -1] = 0 # x8 = 0 always; no washing since we have PFO as carrier phase
    updated_FR = np.concatenate((FR, x_final), axis = 0) ## concat previous and new input parameters
    np.savetxt(path + "//" + "FR.csv", updated_FR, delimiter = ",", fmt="%1.2f") # overwrite it into the existing FR.csv file
    return x_final, updated_FR
## master function
# optimize_options = ["maximize", 0] or ["minimize", y_target]
# policy = "EI" or "PI" or "EPLT"
# def master_code(exp_budget = 75, optimize_options = ["maximize", 0.0], policy = "EI", network_type="cascade", batch_size=8, num_epochs=3000, learning_rate=1e-4, patience=500, min_delta=1e-4, num_models=20, num_objectives=1, num_restarts=100, num_samples=500000, num_selection = 1): # if EI is used
def master_code(exp_budget = 75, optimize_options = ["maximize", 0.0], policy = "EI", network_type="cascade", batch_size=16, num_epochs=3000, learning_rate=1e-3, patience=100, min_delta=1e-4, num_models=20, num_objectives=1, num_restarts=100, num_samples=500000, num_selection = 1): # if EI is used
    loc = input("Please enter the path for the directory that includes files: ")
    print("------------------------------------------------------------------------------------------------------------")
    num_files_old = 0
    while True:
        num_ABS, num_PL = count_files(loc)
        if (num_ABS == num_PL):
            num_files = num_PL
        policy = (policy if num_files < exp_budget else "EPLT")
        if (num_files > num_files_old + num_selection - 1) and (num_files <= exp_budget):
            print("New run starts now:")
            num_files_old = num_files
            WL_ABS, WL_PL, DR_ABS, DR_PL, LR, ABS_all, PL_all, ABS_names, PL_names, FR = read_files(loc)
            y_abs = spectra_extract_ABS(WL_ABS, DR_ABS, LR, ABS_all, ABS_names)
            y_area = spectra_extract_PL(WL_PL, DR_PL, PL_all, PL_names)
            y = np.concatenate((y_abs, y_area), axis = 1)
            new_x, new_y, ensemble, history, x_train, y_train, x_valid, y_valid, structure, obj_fun_best, statNorm_info = exp_sel_botorch(y, FR, optimize_options, policy, network_type=network_type, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate, patience=patience, min_delta=min_delta, num_models=num_models, num_objectives=num_objectives, num_restarts=num_restarts, num_samples=num_samples)
            new_FR, updated_FR = x_dim(new_x, FR, loc)
        elif (num_files > exp_budget):
            print("The experimental budget is reached!")
            break
#         # use this for the test run using the provided dataset 
#         if (num_files_old == 60):
#             break
#     return new_x, new_y, ensemble, history, x_train, y_train, x_valid, y_valid, structure, new_FR, updated_FR, obj_fun_best, statNorm_info

In [ ]:
# new_x, new_y, ensemble, history, x_train, y_train, x_valid, y_valid, structure, new_FR, updated_FR, obj_fun_best, statNorm_info = master_code(exp_budget = 70, optimize_options = ["maximize", 0.0], policy = "EI", network_type="cascade", batch_size=8, num_epochs=3000, learning_rate=1e-4, patience=500, min_delta=1e-6, num_models=20, num_objectives=1, num_restarts=100, num_samples=500000, num_selection = 1)
master_code()